In [47]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Zadanie 3.1

In [48]:
N = 60
s = cv2.imread('../data/lab2-3/pedestrian/input/in000001.jpg')
XX, YY = s.shape[:2]
BUF = np.zeros((XX, YY, N), np.uint8)
iN = 0

In [49]:
path, dirs, files = next(os.walk("../data/lab2-3/pedestrian/input/"))
num_samples = len(files)

#### Średnia

In [56]:
for i in range(300, num_samples):
    I = cv2.imread('../data/lab2-3/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    BUF[:, :, iN] = IG
    iN = (iN + 1) % N

    mean = np.mean(BUF, axis=2).astype(np.uint8)
    diff = cv2.absdiff(mean, IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    cv2.imshow('diff', dilated)
    cv2.waitKey(1)
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [57]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(300, num_samples):
    I = cv2.imread('../data/lab2-3/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    BUF[:, :, iN] = IG
    iN = (iN + 1) % N

    mean = np.mean(BUF, axis=2).astype(np.uint8)
    diff = cv2.absdiff(mean, IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    groundtruth = cv2.imread('../data/lab2-3/pedestrian/groundtruth/gt' + str(i).zfill(6) + '.png')
    groundtruth = cv2.cvtColor(groundtruth, cv2.COLOR_BGR2GRAY)

    TP_matrix = np.logical_and((dilated > 0), (groundtruth > 0))
    TN_matrix = np.logical_and((dilated == 0), (groundtruth == 0))
    FP_matrix = np.logical_and((dilated > 0), (groundtruth == 0))
    FN_matrix = np.logical_and((dilated == 0), (groundtruth > 0))

    TP += TP_matrix.sum()
    TN += TN_matrix.sum()
    FP += FP_matrix.sum()
    FN += FN_matrix.sum()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

print('Precision:', P)
print('Recall:', R)
print('F1:', F1)

Precision: 0.46341843643873004
Recall: 0.8556137111263624
F1: 0.6012092562530225


#### Mediana

In [ ]:
for i in range(300, num_samples):
    I = cv2.imread('../data/lab2-3/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    BUF[:, :, iN] = IG
    iN = (iN + 1) % N

    median = np.median(BUF, axis=2).astype(np.uint8)
    diff = cv2.absdiff(median, IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    cv2.imshow('diff', dilated)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [58]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(300, num_samples):
    I = cv2.imread('../data/lab2-3/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    BUF[:, :, iN] = IG
    iN = (iN + 1) % N

    median = np.median(BUF, axis=2).astype(np.uint8)
    diff = cv2.absdiff(median, IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    groundtruth = cv2.imread('../data/lab2-3/pedestrian/groundtruth/gt' + str(i).zfill(6) + '.png')
    groundtruth = cv2.cvtColor(groundtruth, cv2.COLOR_BGR2GRAY)

    TP_matrix = np.logical_and((dilated > 0), (groundtruth > 0))
    TN_matrix = np.logical_and((dilated == 0), (groundtruth == 0))
    FP_matrix = np.logical_and((dilated > 0), (groundtruth == 0))
    FN_matrix = np.logical_and((dilated == 0), (groundtruth > 0))

    TP += TP_matrix.sum()
    TN += TN_matrix.sum()
    FP += FP_matrix.sum()
    FN += FN_matrix.sum()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

print('Precision:', P)
print('Recall:', R)
print('F1:', F1)

Precision: 0.9529665738648073
Recall: 0.8724676688641839
F1: 0.9109421810367051


## Zadanie 3.2

#### Aproksymacja średniej

In [59]:
I = cv2.imread('../data/lab2-3/pedestrian/input/in000300.jpg')
BG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
alpha = 0.01

for i in range(301, num_samples):
    I = cv2.imread('../data/lab2-3/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    BG = (1 - alpha) * BG + alpha * IG
    diff = cv2.absdiff(BG.astype(np.uint8), IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    cv2.imshow('diff', dilated)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [60]:
TP = 0
TN = 0
FP = 0
FN = 0

I = cv2.imread('../data/lab2-3/pedestrian/input/in000300.jpg')
BG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
alpha = 0.01

for i in range(301, num_samples):
    I = cv2.imread('../data/lab2-3/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    BG = (1 - alpha) * BG + alpha * IG
    diff = cv2.absdiff(BG.astype(np.uint8), IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    groundtruth = cv2.imread('../data/lab2-3/pedestrian/groundtruth/gt' + str(i).zfill(6) + '.png')
    groundtruth = cv2.cvtColor(groundtruth, cv2.COLOR_BGR2GRAY)

    TP_matrix = np.logical_and((dilated > 0), (groundtruth > 0))
    TN_matrix = np.logical_and((dilated == 0), (groundtruth == 0))
    FP_matrix = np.logical_and((dilated > 0), (groundtruth == 0))
    FN_matrix = np.logical_and((dilated == 0), (groundtruth > 0))

    TP += TP_matrix.sum()
    TN += TN_matrix.sum()
    FP += FP_matrix.sum()
    FN += FN_matrix.sum()

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)

print('Precision:', P)
print('Recall:', R)
print('F1:', F1)

Precision: 0.9386583479513319
Recall: 0.8570259896343363
F1: 0.8959866528246562


#### Aproksymacja mediany

In [62]:
I = cv2.imread('../data/lab2-3/pedestrian/input/in000300.jpg')
BG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
alpha = 0.01

for i in range(301, num_samples):
    I = cv2.imread('../data/lab2-3/pedestrian/input/in' + str(i).zfill(6) + '.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    BG[BG < IG] = BG + 1
    BG[BG > IG] = BG - 1
    # BG[BG == IG] = BG
        
    diff = cv2.absdiff(BG.astype(np.uint8), IG)

    T, binarized = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)
    median_filtered = cv2.medianBlur(binarized, 5)
    eroded = cv2.erode(median_filtered, np.ones((3, 3), np.uint8))
    dilated = cv2.dilate(eroded, np.ones((5, 5), np.uint8))

    cv2.imshow('diff', dilated)
    cv2.waitKey(1)
cv2.destroyAllWindows()

TypeError: NumPy boolean array indexing assignment requires a 0 or 1-dimensional input, input has 2 dimensions